In [1]:
import torch
import pandas as pd
import os
import numpy as np
import pydicom as dicom
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import skimage.transform
import torch.optim as optim

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class ConvLayer(nn.Sequential):
    """
    Investigate what we want? -> pooling? Normalisation?
    """
    def __init__(self, in_channel, out_channel, kernel_size=3, stride=1, \
                 groups=1, norm_layer=None) -> None:
        padding = (kernel_size - 1) // 2
        if norm_layer == None:
            norm_layer = nn.BatchNorm2d
        
        super(ConvLayer, self).__init__(
            nn.Conv2d(in_channel, out_channel, kernel_size=kernel_size, \
                      stride=stride, padding=padding, groups=groups, bias= False),
            norm_layer(out_channel),
            nn.ReLU6(inplace=True)
        )

In [ ]:
# Forget this...
class InvertedResidual(nn.Module):
    def __init__(self, in_channel, out_channel, stride, expand_ratio, \
                 norm_layer=None) -> None:
        super(InvertedResidual, self).__init__()
        hidden_channel = in_channel * expand_ratio
        self.use_shortcut = stride == 1 and in_channel == out_channel
        if norm_layer == None:
            norm_layer = nn.BatchNorm2d
            
        # Investigate if this is necessary? Maybe if architecture is too big?
        
        
            
        
        

In [ ]:
# Forget this...
class CNN(nn.Module):
    """
    Maybe Replace BatchNorm with AdaptiveAvg
    """
    DROPOUT_RATE = .0
    def __init__(self, output_dim = 4) -> None: 
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            nn.Dropout(p=DROPOUT_RATE)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=(3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            nn.Dropout(p=DROPOUT_RATE)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            nn.Dropout(p=DROPOUT_RATE)
        )
        
        self.layer4 = nn.Sequential(
            nn.Linear(128*3*3, 625, bias=True),
            nn.ReLU(),
            nn.Dropout(p=DROPOUT_RATE)
        )
        
        self.output_layer = nn.Linear(625, output_dim, bias=True)
        
        self.log_softmax = F.logsoftmax
        
    def forward(self, x: torch.Tensor): #-> Something? torch.tensor I think?
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(torch.flatten(x, start_dim=1))
        x = self.output_layer(x)

In [7]:
class CNN(nn.Module): # better version lol
    def __init__(self, output_dim = 5) -> None:
        DROPOUT_RATE = .2
        super(CNN, self).__init__()
        expansions = [32, 64, 128]
        features = []
        input_channel, last_channel = 1, 1
        for channel in expansions:
            output_channel = channel
            features.append(ConvLayer(input_channel, output_channel))
            input_channel = output_channel
        
        dense_layer = [
            nn.Flatten(start_dim=1),
            nn.Linear(expansions[-1]*28*28, last_channel, bias=True),
            nn.ReLU(),
            nn.Dropout(p=DROPOUT_RATE)
        ]
        
        classification_layer = [
            nn.Linear(last_channel, output_dim, bias=True),
        ]
            
        features.extend(dense_layer)
        features.extend(classification_layer)
                
        self.conv = nn.Sequential(*features)
        self.log_softmax = F.log_softmax # investigate quantisable version of this # Maybe do this in PYNQ instead?
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        return self.log_softmax(x, dim=1)


In [10]:
model = CNN(output_dim=4)
model(torch.rand(1, 1, 28, 28, requires_grad=True))

tensor([[-1.6225, -1.9260, -1.1464, -1.0815]], grad_fn=<LogSoftmaxBackward0>)

Forget What's Below Here...

In [5]:
import io
import torch.utils.model_zoo as model_zoo
import torch.onnx

In [6]:
torch.onnx.export(model, torch.rand(1, 1, 28, 28, requires_grad=True), "CNN_Test.onnx", \
                  export_params=True, opset_version=10, do_constant_folding=True, \
                 input_names = ['input'], output_names = ['output'])

In [7]:
import onnx

onnx_model = onnx.load("CNN_Test.onnx")
onnx.checker.check_model(onnx_model)